In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#Read 3rd sheet of excel files
df2015 = pd.read_excel('../../Data/EnergieUebersichtCH-2015.xls', sheet_name=2)
df2016 = pd.read_excel('../../Data/EnergieUebersichtCH-2016.xls', sheet_name=2)
df2017 = pd.read_excel('../../Data/EnergieUebersichtCH-2017.xls', sheet_name=2)
df2018 = pd.read_excel('../../Data/EnergieUebersichtCH-2018.xls', sheet_name=2)
df2019 = pd.read_excel('../../Data/EnergieUebersichtCH-2019.xls', sheet_name=2)
df2020 = pd.read_excel('../../Data/EnergieUebersichtCH-2020.xlsx', sheet_name=2)
df2021 = pd.read_excel('../../Data/EnergieUebersichtCH-2021.xlsx', sheet_name=3)
df2022 = pd.read_excel('../../Data/EnergieUebersichtCH-2022.xlsx', sheet_name=3)
df2023 = pd.read_excel('../../Data/EnergieUebersichtCH-2023.xlsx', sheet_name=3)

In [ ]:
# Put all the dataframes in a list
df_list = [df2015, df2016, df2017, df2018, df2019, df2020, df2021, df2022, df2023]

# Print the number of columns of each dataframe
for df in df_list:
    print(df.shape[1])

In [ ]:
# Concatenate all the dataframes
dffull = pd.concat(df_list, axis=0, ignore_index=True)
print(dffull.shape)
dffull.head()

In [ ]:
df_prod = pd.read_csv('../../Data/15min_prod.csv', sep=',')
#Remove the first row (kwh)
df_prod = df_prod.iloc[1:]

df_cons = pd.read_csv('../../Data/15min_cons.csv', sep=',')
#Remove the first row (kwh)
df_cons = df_cons.iloc[1:]

df_prod.head()

In [ ]:
#Find rows where first column is 'Zeitstempel' and remove them
df_prod = df_prod.drop(df_prod[df_prod['Date'].str.contains('Zeitstempel')].index)
df_cons = df_cons.drop(df_cons[df_cons['Date'].str.contains('Zeitstempel')].index)

#Reset index
df_prod = df_prod.reset_index(drop=True)
df_cons = df_cons.reset_index(drop=True)


In [ ]:
#Convert all the columns but the first to float
df_prod.iloc[:,1:] = df_prod.iloc[:,1:].astype(float)

#Convert all the columns but the first to float
df_cons.iloc[:,1:] = df_cons.iloc[:,1:].astype(float)

In [ ]:
#If 'Date' column does not follow the format 'MM.DD.YYYY HH:MM', transform it from 'YYYY-MM-DD HH:MM:SS' to 'DD.MM.YYYY HH:MM'
for i in range(len(df_cons)):
    if len(df_cons['Date'][i]) > 16:
        df_cons['Date'][i] = df_cons['Date'][i][8:10] + '.' + df_cons['Date'][i][5:7] + '.' + df_cons['Date'][i][0:4] + ' ' + df_cons['Date'][i][11:16]
df_cons.head()

#If 'Date' column does not follow the format 'MM.DD.YYYY HH:MM', transform it from 'YYYY-MM-DD HH:MM:SS' to 'DD.MM.YYYY HH:MM'
for i in range(len(df_prod)):
    if len(df_prod['Date'][i]) > 16:
        df_prod['Date'][i] = df_prod['Date'][i][8:10] + '.' + df_prod['Date'][i][5:7] + '.' + df_prod['Date'][i][0:4] + ' ' + df_prod['Date'][i][11:16]
df_prod.head()

In [ ]:
df_cons.head()

In [ ]:
#Convert the date column to datetime
print(df_cons.columns)
df_cons['Date'] = pd.to_datetime(df_cons['Date'], format='%d.%m.%Y %H:%M')
df_cons.head()

#Convert the date column to datetime
print(df_prod.columns)
df_prod['Date'] = pd.to_datetime(df_prod['Date'], format='%d.%m.%Y %H:%M')

In [ ]:
#Resample the data to hourly
df_cons = df_cons.resample('H', on='Date').sum()
df_cons.head()

#Resample the data to hourly
df_prod = df_prod.resample('H', on='Date').sum()
df_prod.head()

In [ ]:
df_cons = df_cons.reset_index()
df_cons.head()

df_prod = df_prod.reset_index()
df_prod.head()

In [ ]:
df_cons.head()

In [ ]:
df_testt = {}
df_testt[0] = pd.DataFrame()
df_testt[0]['Production'] = df_prod.iloc[:,1]

df_testt[0].head()

In [ ]:
df_temp = {}

for i in range (1,19):
    df_temp[i-1] = pd.DataFrame()
    df_temp[i-1]['Date'] = df_prod.iloc[:,0] 
    df_temp[i-1]['Production'] = df_prod.iloc[:,i] 
    df_temp[i-1]['Consumption'] = df_cons.iloc[:,i]
    df_temp[i-1]['Canton'] = df_cons.columns[i].split()[-1]

df_temp[1].head()

In [ ]:
#Concate all the dataframes vertically
df_concat = pd.concat(df_temp, axis=0, ignore_index=True)
df_concat.head()

In [ ]:
#Sort the dataframe by date
df_concat = df_concat.sort_values(by=['Date'])

#Reset index
df_concat = df_concat.reset_index(drop=True)

df_concat.head()

In [ ]:
df_concat.to_csv('../../Data/1hour_concat.csv', index=False)

In [ ]:
#Plot the consumption of all cantons using plotly Canton as color
import plotly.express as px
fig = px.line(df_concat, x="Date", y="Consumption", color='Canton')
fig.show()

In [ ]:
fig = px.line(df_concat, x="Date", y="Production", color='Canton')
fig.show()